<img src="../imgs/Adevinta-ULPGC-logo.jpg" width="530px" align="right">

# **Aprendizaje por refuerzo 4**

Hasta ahora hemos visto los conceptos y algoritmos clásicos del aprendizaje por refuerzo, pero cuando los problemas se vuelven más complejos y el número de estados se vuelve inmanejable, es necesario recurrir a técnicas más avanzadas. Vermos cómo los algoritmos clásicos, como el Q-Learning, pueden ser mejorados con redes neuronales profundas para manejar problemas más complejos.

## **Deep Q-Learning**

El algoritmo Deep Q-Learning (DQL) es una técnica de aprendizaje por refuerzo que combina la arquitectura de las redes neuronales profundas con el método de Q-learning. Esta combinación permite que el algoritmo sea eficaz en entornos con estados de alta dimensión que serían difíciles de manejar con técnicas más tradicionales. Deep Q-Learning fue popularizado por DeepMind en 2013, especialmente con su éxito al entrenar modelos que podían jugar a varios juegos de Atari directamente desde los píxeles de entrada.

<br>
<div align="center">
    <img src="imgs/deep_q_learning_pong.gif" width="500px">
</div>

https://www.youtube.com/watch?v=TmPfTpjtdgg



### **¿Cómo funciona?**

En primer lugar vamos a que el conjunto de estados corresponde a la pantalla del juego, que es una imagen de 210x160 píxeles con 3 canales de color (RGB). Por lo tanto, el espacio de estados es de 210x160x3 = 100800 dimensiones. Esto significa que el espacio de estados es extremadamente grande y sería imposible almacenar una tabla de valores Q para cada estado. En su lugar, utilizamos una red neuronal profunda para aproximar la función Q.

Recordemos que la función Q (que vimos anteriormente) es una función que toma un estado y una acción y devuelve un valor que representa la recompensa esperada al tomar esa acción en ese estado. En el caso del DQL, la función Q es aproximada por una red neuronal profunda, pero no toma como entrada el estado y la acción, sino solo el estado devolviendo un vector de valores Q, uno para cada acción posible. La acción que se toma es la que tiene el valor Q más alto. 

<br>
<div align="center">
    <img src="imgs/deep.jpg" width="500px">
</div>

En el caso del ejemplo basta con una salida binaria, ya que solo hay dos posibles acciones: derecha o izquierda.

<br>
<div align="center">
    <img src="imgs/pong_deepQL.svg" width="700px">
</div>

En el caso del juego de Atari que se ilustra arriba, una red neuronal con capas inicialmente convolutivas y luego densas se encarga de procesar la imagen de entrada y devolver un vector de valores Q (en este caso particular solo un valor). Pero, ¿en qué se transforma ahora la tabla Q? En lugar de almacenar una tabla de valores Q, la red neuronal se entrena para predecir los valores Q. La tabla pasa a ser ahora una función, ya no es necesario almacenar todos los valores Q, sino que la red neuronal los calcula en función del estado de entrada. Recordemos cómo se actualizaba la tabla Q en el algoritmo Q-Learning:

$$Q(s, a) = r + \gamma \max_{a'} Q(s', a') \tag{1}$$

Observa que podríamos modificar la fórmula $(1)$ para que la actualización de Q se haga de forma conservadora, es decir, haya una proporción $\alpha$ entre el valor actual y el nuevo valor. Esto puede ser útil en los casos en que tengamos escenarios donde las recompensas puedan ser variables y no queramos que el valor Q cambie de forma brusca. La fórmula $(1)$ se puede reescribir como:

$$Q(s, a) = (1 - \alpha) Q(s, a) + \alpha \left( r + \gamma \max_{a'} Q(s', a') \right) \tag{2}$$

Ahora, mediante la propiedad distributiva, podemos reescribir la fórmula $(2)$ como:

$$Q(s, a) = Q(s, a) + \alpha \left( r + \gamma \max_{a'} Q(s', a') - Q(s, a) \right)  \tag{3}$$

Fíjate que ahora, cuando la función Q converja, $r + \gamma \max_{a'} Q(s', a') \approx Q(s, a)$. Esto se parece mucho a la fórmula de la función de pérdida de una red neuronal, donde el objetivo es minimizar la diferencia entre el valor predicho y el valor real. En este caso, el valor real (label) podría ser $r + \gamma \max_{a'} Q(s', a')$ y el valor predicho $Q(s, a)$.



<div style="background-color: #fdebd0; border-left: 5px solid #ffb366; padding: 1.5em; margin: 30px; width: 600px">
<b>Deep Q-Learning</b> utiliza una red neuronal para aproximar, dado un estado (la pantalla del juego en un instante dado), los diferentes valores Q para cada acción posible en ese estado.
</div>

Vamos a empezar la transición de Q-Learning a Deep Q-Learning. Lo primero que vamos a hacer es definir una red que llamaremos $Q$ y otra red idéntica que llamaremos $Q_{\text{target}}$. La red $Q$ será la que utilizaremos para aproximar la función Q y la red $Q_{\text{target}}$ será una copia de la red $Q$ que utilizaremos para calcular el valor de la recompensa esperada en la fórmula $(3)$. La red $Q_{\text{target}}$ se actualizará cada cierto número de pasos con los pesos de la red $Q$.

### **Pasos del algoritmo**

1. **Inicialización**: Antes de comenzar el entrenamiento, se inicializan los siguientes componentes:
   - **Memoria de repetición (memory replay)**: un almacenamiento de transiciones previas, cada una compuesta por el estado actual, la acción tomada, la recompensa obtenida, y el siguiente estado.
   - **Red de acción-valor (Q)**: una red neuronal profunda que estima el valor de acción $ Q(s, a; \theta) $ para un estado $ s $ y una acción $ a $, con parámetros $ \theta $.
   - **Red de acción-valor objetivo ($ Q_{target} $)**: una copia de la red Q que se utiliza para generar las estimaciones de valor objetivo durante las actualizaciones, con parámetros $ \theta^- $.

2. **Proceso por episodios**: El entrenamiento se realiza sobre varios episodios, donde un episodio representa una secuencia de estados, acciones y recompensas hasta que se alcanza un estado terminal o se cumple una condición de parada.

3. **Selección de acciones**:
   - Seleccionar acciones a través de una política $ \epsilon $-greedy, que elige una acción aleatoria con probabilidad $ \epsilon $ para explorar el espacio de acciones, o la acción con el mayor valor Q estimado para el estado actual $ s $ con probabilidad $ 1 - \epsilon $.

4. **Interacción con el entorno**:
   - Ejecutar la acción seleccionada en el emulador o entorno, observar la recompensa resultante y el siguiente estado.

5. **Almacenamiento de experiencias**:
   - Almacenar la transición (estado, acción, recompensa, siguiente estado) en la memoria de repetición.

6. **Muestreo de experiencias**:
   - Muestrear un minibatch aleatorio de transiciones de la memoria de repetición para el aprendizaje, reduciendo así la correlación entre las muestras y permitiendo un aprendizaje más eficiente.

7. **Cálculo del valor objetivo**:
   - Para cada muestra en el minibatch, calcular el valor objetivo $ y_j $ que la red debe aprender a predecir. Si el siguiente estado es terminal, el valor objetivo es simplemente la recompensa observada. De lo contrario, el valor objetivo se calcula como la recompensa observada más el valor máximo $Q_{target}$ estimado para el siguiente estado, descontado por el factor de descuento $ \gamma $.

8. **Entrenamiento de la red Q**:
   - Realizar una actualización de descenso de gradiente en la red $Q$ con el propósito de minimizar la diferencia entre los valores $Q_{target}$ estimados actuales y los valores objetivo $ y_j $.

9. **Actualización de la red objetivo**:
   - Cada cierto número de pasos, copiar los parámetros de la red $Q$ a la red de acción-valor objetivo $Q_{target}$ para estabilizar el aprendizaje.

10. **Repetir el proceso**:
    - Repetir los pasos 3 a 9 para cada paso de tiempo dentro de un episodio, y para cada episodio.

11. **Terminación**:
    - El proceso continúa hasta que se cumple un criterio de terminación, como un número fijo de episodios o una mejora insuficiente en la recompensa total obtenida.

Este proceso permite que el agente aprenda políticas de acción que maximicen la recompensa total en un entorno, basándose en la experiencia acumulada y utilizando la generalización proporcionada por las redes neuronales profundas para manejar grandes espacios de estado.